# Special Cases - Shoe Manufacturing Example

Suppose that we have three machines to manufacture shoes. The cost to operate each machine is represented as

<center>$C_1 = 0.01n_1^2 + 2 n_1$


$C_2=6n_2$

$C_3=7n_3$</center>

where $n_1,n_2,n_3$ are the number of shoes manufactured by each machine. Each machine has a maximum production capacity of 1000 shoes. To facilitate a total production of 2100 shoes, how many shoes should each machine make to minimize the cost?

In [ ]:
! pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatibl

In [ ]:
from ortools.sat.python import cp_model

In [ ]:
model = cp_model.CpModel()

n1 = model.NewIntVar(0, 1000, 'n1')
n2 = model.NewIntVar(0, 1000, 'n2')
n3 = model.NewIntVar(0, 1000, 'n3')

# Introduce a new variable for the squared term and add a constraint
n1_squared = model.NewIntVar(0, 1000*1000, 'n1_squared') # Max value is 1000^2
model.AddMultiplicationEquality(n1_squared, [n1, n1])

model.Add(n1 + n2 + n3 == 2100)

# Use the new variable in the objective function
model.minimize(0.01*n1_squared + 2*n1 + 6*n2 + 7*n3)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Optimal cost: {solver.ObjectiveValue()}')
    print(f'n1: {solver.Value(n1)}')
    print(f'n2: {solver.Value(n2)}')
    print(f'n3: {solver.Value(n3)}')
else:
    print('No solution found.')

Optimal cost: 13075.0
n1: 250
n2: 1000
n3: 850


## Interdependent Costs

Suppose that we have three machines to manufacture shoes. The cost to operate each machine is now represented as

<center>$C_1 = 0.01n_1^2 + 2 n_1$


$C_2=6n_1n_2$

$C_3=7n_3$</center>

where $n_1,n_2,n_3$ are the number of shoes manufactured by each machine. Each machine has a maximum production capacity of 1000 shoes. To facilitate a total production of 2100 shoes, how many shoes should each machine make to minimize the cost?

In [ ]:
model = cp_model.CpModel()

n1 = model.NewIntVar(0, 1000, 'n1')
n2 = model.NewIntVar(0, 1000, 'n2')
n3 = model.NewIntVar(0, 1000, 'n3')

# Introduce a new variable for the squared term and add a constraint
n1_squared = model.NewIntVar(0, 1000*1000, 'n1_squared') # Max value is 1000^2
model.AddMultiplicationEquality(n1_squared, [n1, n1])
n1_n2 = model.NewIntVar(0, 1000*1000, 'n1_n2')
model.AddMultiplicationEquality(n1_n2, [n1, n2])

model.Add(n1 + n2 + n3 == 2100)

# Use the new variable in the objective function
model.minimize(0.01*n1_squared + 2*n1 + 6*n1_n2 + 7*n3)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Optimal cost: {solver.ObjectiveValue()}')
    print(f'n1: {solver.Value(n1)}')
    print(f'n2: {solver.Value(n2)}')
    print(f'n3: {solver.Value(n3)}')
else:
    print('No solution found.')

Optimal cost: 607300.0
n1: 100
n2: 1000
n3: 1000


## Introducing a Boolean Constraint

Suppose that we have three machines to manufacture shoes. Machine 2 has a startup cost of $1000. The cost to operate each machine is now represented as

<center>$C_1 = 2n_1$


$C_2=b\times(1000 + 6n_2)\space\space$  where $b$ is a binary variable taking on values of 0 or 1.

$C_3=7n_3$

$n_1,n_3 \le 1000$

$n_2 \le 1000\times b$</center>



where $n_1,n_2,n_3$ are the number of shoes manufactured by each machine. To facilitate a total production of 2100 shoes, how many shoes should each machine make to minimize the cost?

In [ ]:
model = cp_model.CpModel()

n1 = model.NewIntVar(0, 1000, 'n1')
n2 = model.NewIntVar(0, 1000, 'n2')
n3 = model.NewIntVar(0, 1000, 'n3')
b = model.NewBoolVar('b')

model.Add(n1 + n2 + n3 == 2100)

# Introduce a new variable for the cost of machine 2
cost2 = model.NewIntVar(0, 1000 + 6 * 1000, 'cost2') # Max cost of machine 2

# Add constraints to link cost2 to b and n2 using OnlyEnforceIf

# If b is 1, cost2 = 1000 + 6 * n2
model.Add(cost2 == 1000 + 6 * n2).OnlyEnforceIf(b)

# If b is 0, cost2 = 0
model.Add(cost2 == 0).OnlyEnforceIf(b.Not())

# Add constraint to ensure n2 is 0 if b is 0 (machine 2 is not used)
model.Add(n2 == 0).OnlyEnforceIf(b.Not())

# Use the new variable in the objective function
model.minimize(2*n1 + cost2 + 7*n3)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Optimal cost: {solver.ObjectiveValue()}')
    print(f'n1: {solver.Value(n1)}')
    print(f'n2: {solver.Value(n2)}')
    print(f'n3: {solver.Value(n3)}')
    print(f'b: {solver.Value(b)}')
else:
    print('No solution found.')

Optimal cost: 9700.0
n1: 1000
n2: 1000
n3: 100
b: 1


## (Binary) * (Binary) Example

Suppose that we have three machines to manufacture shoes. Machine 2 can only be operating if Machine 1 is operating. The cost to operate each machine is now represented as

<center>$C_1 = b_12n_1$

$C_2=b_1b_26n_2$

$C_3=7n_3$

$n_1 \le b_1 \times 1000$

$n_2 \le b_1b_2 \times 1000$

$n_3 \le 1000$</center>



where $n_1,n_2,n_3$ are the number of shoes manufactured by each machine. $b_0$ indicates whether Machine 1 will be used and $b_0b_1$ indicates whether Machine 2 will be used.  To facilitate a total production of 2100 shoes, how many shoes should each machine make to minimize the cost?

In [15]:
model = cp_model.CpModel()

n1 = model.NewIntVar(0, 1000, 'n1')
n2 = model.NewIntVar(0, 1000, 'n2')
n3 = model.NewIntVar(0, 1000, 'n3')
b1 = model.NewBoolVar('b1')
b2 = model.NewBoolVar('b2')

model.Add(n1 + n2 + n3 == 2100)

# Define the relationship b1_and_b2 == b1 AND b2 using implications
b1_and_b2 = model.NewBoolVar('b1_and_b2')
model.AddImplication(b1, b1_and_b2).OnlyEnforceIf(b2) # If b1 is true and b2 is true, then b1_and_b2 is true
model.AddImplication(b1_and_b2, b1) # If b1_and_b2 is true, then b1 is true
model.AddImplication(b1_and_b2, b2) # If b1_and_b2 is true, then b2 is true


# Constraints to link production to binary variables
# If b1 is 0, n1 must be 0
model.Add(n1 == 0).OnlyEnforceIf(b1.Not())
# If b1 is 1, n1 can be up to 1000 (this is already covered by the variable definition)


# Machine 2 can only operate if Machine 1 operates (b1_and_b2 is true)
# If b1_and_b2 is 0, n2 must be 0
model.Add(n2 == 0).OnlyEnforceIf(b1_and_b2.Not())
# If b1_and_b2 is 1, n2 can be up to 1000 (this is already covered by the variable definition)


model.Add(n3 <= 1000) # Capacity constraint for n3


# Use the production variables directly in the objective function with costs
model.minimize(2*n1 + 6*n2 + 7*n3)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Optimal cost: {solver.ObjectiveValue()}')
    print(f'n1: {solver.Value(n1)}')
    print(f'n2: {solver.Value(n2)}')
    print(f'n3: {solver.Value(n3)}')
    print(f'b1: {solver.Value(b1)}')
    print(f'b2: {solver.Value(b2)}')
    print(f'b1_and_b2: {solver.Value(b1_and_b2)}')

else:
    print('No solution found.')

Optimal cost: 8700.0
n1: 1000
n2: 1000
n3: 100
b1: 1
b2: 1
b1_and_b2: 1
